# Differential gene expression analysis

Exercise for analysing gene expression changes between conditions.

* **Contact:** mate.balajti@unibas.ch

#### Differential gene expression analysis

One of the most common experiment types making use of RNA-Seq is a differential
gene expression analysis where gene expression levels across different
conditions (e.g., healthy vs disease) or compared to each other. This type of
experiment is often the basis for discovering genes that are relevant to a
given physiological or disease process. Following this relatively open-ended
discovery phase of a study, a list of genes with a strikingly different
expression pattern is often the basis for further, more detailed mechanistic
studies.

Required files:

* Control mate 1: `control.mate_1.fq.gz` (from Exercise 2)
* Control mate 2: `control.mate_2.fq.gz` (from Exercise 2)
* Treated mate 1: `treated.mate_1.fq.gz`
* Treated mate 2: `treated.mate_2.fq.gz`

Procedure:

* Create a table of counts for each gene (rows) and sample (columns)
* Analyze gene expression across different conditions based on such a count
  table

#### Exercise 3.1: Align the treatment reads to the reference genome (2 points)

Align the treated sample’s paired-end FASTQ files to the reference genome with `STAR` (similarly to Exercise 2) to produce a sorted, indexed `BAM` suitable for counting. 

Required files:

* Genome index (same one used for control in Exercise 2, created from `Mus_musculus.GRCm38.dna_rm.chr19.fa.gz`)
* Annotation GTF (same one used for control in Exercise 2, `Mus_musculus.GRCm38.88.chr19.gtf.gz`)
* Treated reads: `treated.mate_1.fq.gz`, `treated.mate_2.fq.gz`

Run `STAR` on the treated read pair using the same parameters and genome index you used for control.

> When using STAR, ensure `--outSAMtype BAM SortedByCoordinate` to get sorted BAM files as output; otherwise sort after alignment (using `samtools`)

In [ ]:
### Copy your command here

STAR \
    --genomeDir STAR_index_chr19 \
    --readFilesIn treated.mate_1.fq.gz treated.mate_2.fq.gz \
    --readFilesCommand zcat \
    --runThreadN 4 \
    --outFileNamePrefix treated_ \
    --outSAMtype BAM SortedByCoordinate \
    --outSAMunmapped Within \
    --outSAMattributes Standard \
    --quantMode GeneCounts \
    --sjdbGTFfile Mus_musculus.GRCm38.88.chr19.gtf

#### Exercise 3.2: Generate gene-level counts (3 points)

Summarize aligned reads to gene counts for control and treated using the GTF annotation. 
Choose a counting tool: **featureCounts** (recommended) or HTSeq-count.

featureCounts is part of the [subread](https://subread.sourceforge.net/) package (available through [bioconda](https://anaconda.org/bioconda/subread)). 
Follow [featureCounts\' manual](https://subread.sourceforge.net/SubreadUsersGuide.pdf#section.6.2) to summarize the aligned reads from the `BAM` files into a gene × sample count table (rows = genes, columns = control, treated).

Input files:
* Output from Exercise 2 and 3.1: `control.sorted.bam`, `treated.sorted.bam`
* Annotation GTF: `Mus_musculus.GRCm38.88.chr19.gtf.gz`


> Ensure both BAMs are coordinate-sorted and indexed before counting.

> Note that we're dealing with paired-end reads, therefore you have to count _fragments_ instead of reads (`-p` and `--countReadPairs` params are required).

In [ ]:
### Copy your command here
featureCounts \
  -a Mus_musculus.GRCm38.88.chr19.gtf \
  -o gene_counts.txt \
  -p --countReadPairs \
  -T 4 \
  -g gene_id \
  control.sorted.bam treated_Aligned.sortedByCoord.out.bam


#### Exercise 3.3: DESeq2 size-factor normalization (3 points)

Because we only have __1 control__ and __1 treated__ sample, we cannot estimate biological variance across replicates and therefore cannot perform reliable hypothesis testing. To keep the exercise scientifically grounded yet minimal, we will:

1. Implement DESeq2’s size-factor normalization
    - Compute the geometric mean per gene across samples (only genes with all non-zero counts)
    - For each sample, compute gene-wise ratios count / geometric mean
    - The size factor is the median of these ratios; divide raw counts by the size factor to obtain normalized counts

2. Rank genes by log2FC (and optionally by absolute log2FC)

3. Visualize the results
    - MA plot of (mean normalized count) vs (log2FC) for an effect-size view.

In a real analysis with adequate replication, these steps would be followed by __dispersion estimation__, __NB-GLM fitting__ under an appropriate design (accounting for covariates), __Wald/LRT__ testing with FDR control, and __LFC shrinkage__, plus thorough QC (PCA, dispersion plots, Cook’s/outlier checks). If you're interested, you can read more about DESeq2 [here](https://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html).



In [25]:
import pandas as pd
import numpy as np

# Read in your featureCounts output file into a pandas df
FEATURECOUNTS_TSV = "gene_counts.txt"  # path to your featureCounts output

fc = pd.read_csv(FEATURECOUNTS_TSV, sep="\t", comment="#").set_index("Geneid")


# Identify sample columns (featureCounts puts them at the end and they often end with .bam)
# Create the df "counts_df" with the sample columns and "Geneid" as the index
# You can change the name of the headers as well
counts_df = fc.iloc[:, -2:]
counts_df.columns = ['control', 'treated']



print(counts_df.head())

                    control  treated
Geneid                              
ENSMUSG00000100969        0        0
ENSMUSG00000093983        0        0
ENSMUSG00000024831      253      215
ENSMUSG00000024829      410      362
ENSMUSG00000099261        0        0


Implement the function `deseq_size_factors()` that takes one argument, a dataframe with two numerical columns (the counts from the samples) and returns a `pd.Series` of DESeq median-of-ratios size factors (indexed by the column names): compute per-gene geometric means, compute per-sample ratios `count/gm`, take the median ratio per sample as its size factor, and return the Series named `size_factor`.

> **Note:**
>
> If either samples have 0 count for a gene, the GM will be 0. Therefore, either filter out such genes, or when computing the median, ignore `NaN`'s (look it up in `numpy` how it's done).

In [28]:
import numpy as np
import pandas as pd

def deseq_size_factors(counts_df: pd.DataFrame) -> pd.Series:
    """
    DESeq median-of-ratios normalization:
      1) geometric mean (gm) per gene across samples
      2) for each sample, compute gene-wise ratios count / gm
      3) size factor = median of ratios over genes with gm > 0
    """
    geometric_means = np.exp(np.mean(np.log(counts_df + 1), axis = 1))
    filtered_genes = geometric_means > 0

    ratios = counts_df.loc[filtered_genes].divide(geometric_means[filtered_genes], axis=0)
    size_factors = ratios.median(axis=0)

    return size_factors 
    










size_factors = deseq_size_factors(counts_df)
print(size_factors)


control    0.408248
treated    0.633257
dtype: float64


Then, divide the counts by the computed size factors to get the normalized counts.

In [31]:
normalized_df = counts_df.divide(size_factors, axis=1)
print(normalized_df.head())

                        control     treated
Geneid                                     
ENSMUSG00000100969     0.000000    0.000000
ENSMUSG00000093983     0.000000    0.000000
ENSMUSG00000024831   619.720905  339.514418
ENSMUSG00000024829  1004.290795  571.647531
ENSMUSG00000099261     0.000000    0.000000


#### Exercise 3.4: Calculate fold change and explore (1 point)

Calculate the log2 fold change between the treated sample and the control sample from your normalized data. 

In [37]:
# Write your code here
fold_changes = np.log2((normalized_df['treated'] + 1) / (normalized_df['control'] + 1))
normalized_df['log2_fc'] = fold_changes
sorted_norm_df = normalized_df.sort_values(by='log2_fc', ascending= False)
print(sorted_norm_df.head(3))
print(sorted_norm_df.tail(3))




                      control      treated   log2_fc
Geneid                                              
ENSMUSG00000067577   0.000000    91.589936  6.532783
ENSMUSG00000024669  19.595918  1664.410214  6.337375
ENSMUSG00000045903   0.000000    18.949642  4.318291
                         control      treated   log2_fc
Geneid                                                 
ENSMUSG00000043789    776.488248    56.848926 -3.748459
ENSMUSG00000025014  16536.505254  1080.129589 -3.935130
ENSMUSG00000025082     63.686733     1.579137 -4.648510


Which genes are up- and downregulated when the sample undergoes treatment? Write the 3 genes with the highest `log2FC` and 3 genes with the lowest `log2FC`. Look up the gene names that belong to these 6 Gene ID's. What are their functions?

> **Hint:** 
>
> You could make use of the [Ensembl](https://www.ensembl.org/index.html) or [NCBI](https://www.ncbi.nlm.nih.gov/) databases. 

In [ ]:
# Write your answers here
"""
highest: 
ENSMUSG00000067577 RIKEN cDNA A430093F15 long noncoding RNA
ENSMUSG00000024669 CD5 antigen, protein coding, involved in T cell costimulation and apoptotic signaling pathway
ENSMUSG00000045903 neuronal PAS domain protein 4, protein coding, Enables DNA-binding transcription factor activity

lowest: 
ENSMUSG00000043789 von Willebrand factor C and EGF domains, protein coding, Predicted to enable calcium ion binding activity and to be involved in the cellular response to virus
ENSMUSG00000025014 deoxynucleotidyltransferase, terminal protein coding, Enables DNA nucleotidylexotransferase activity
ENSMUSG00000025082 von Willebrand factor A domain containing 2, protein coding, Enables identical protein binding activity.


"""

#### Exercise 3.5: Visualize results (1 point)

Use the `matplotlib` or [`seaborn`](https://seaborn.pydata.org/) Python package to create an MA-plot showing your results. The plot shows the average expression on the x-axis and the log2FC on the y. It should look something like this:

![MA-plot](ma_plot.png)

In [ ]:
# Write your code here
import matplotlib.pyplot as plt







plt.show()
